### Miscellaneous


In [27]:
%run core.ipynb

#### Turing Machines

In [28]:
from array import array


class Tape:

    def __init__(self, values):
        self.values = array("I", values)
        self.pos = 0
        
    def __repr__(self):
        return " ".join([f"{v}" for v in self.values[:self.pos]] + 
                        [_blu(self.values[self.pos])] + 
                        [f"{v}" for v in self.values[self.pos + 1:]])
    
    def __getitem__(self, i):
        return self.values[i]
    
    def read(self):
        return self.values[self.pos]

    def write(self, v):
        self.values[self.pos] = v

    def move(self, mv):
        if mv == "L":
            if self.pos == 0:
                self.values.insert(0, 0)
            else:
                self.pos -= 1
        else:
            if self.pos == len(self.values) - 1:
                self.values.append(0)
            self.pos += 1


In [42]:
class TuringMachine:
    
    def __init__(self, instructions):
        self.instructions = instructions
        
    def __getitem__(self, i):
        return self.instructions[i]

    def __call__(self, tape, max_iterations=100):
        i, instr, mv = 0, 0, None
        while not mv == "S":
            if i > max_iterations:
                raise StopIteration(f"max iterations {max_iterations} reached")
            value = tape.read()
            next_instr, new_value, mv = self.instructions[instr][value]
            yield instr, next_instr, value, new_value, mv
            instr = next_instr
            tape.write(new_value)
            tape.move(mv)
            i += 1


In [43]:
def run_turing_machine(tm, tape):
    for instr, _, value, new_value, move in tm(tape):
        print(f"{instr}: {value} -> {new_value} mv={move}:", tape)
    print()
    print(f"result: {tape}")

In [44]:
UNARY_PLUS_ONE = {
    0: {
        0: (0, 0, "R"), 
        1: (1, 1, "R"),
    },
    1: {
        0: (0, 1, "S"), 
        1: (1, 1, "R"),
    },
}
unary_plus_one = TuringMachine(UNARY_PLUS_ONE)

In [45]:
tape = Tape([0, 0, 1, 1, 1, 0, 0])
run_turing_machine(unary_plus_one, tape)

0: 0 -> 0 mv=R: 0 0 1 1 1 0 0
0: 0 -> 0 mv=R: 0 0 1 1 1 0 0
0: 1 -> 1 mv=R: 0 0 1 1 1 0 0
1: 1 -> 1 mv=R: 0 0 1 1 1 0 0
1: 1 -> 1 mv=R: 0 0 1 1 1 0 0
1: 0 -> 1 mv=S: 0 0 1 1 1 0 0

result: 0 0 1 1 1 1 0


In [46]:
tape = Tape([0, 1, 1, 1, 1, 1, 0, 0])
run_turing_machine(unary_plus_one, tape)

0: 0 -> 0 mv=R: 0 1 1 1 1 1 0 0
0: 1 -> 1 mv=R: 0 1 1 1 1 1 0 0
1: 1 -> 1 mv=R: 0 1 1 1 1 1 0 0
1: 1 -> 1 mv=R: 0 1 1 1 1 1 0 0
1: 1 -> 1 mv=R: 0 1 1 1 1 1 0 0
1: 1 -> 1 mv=R: 0 1 1 1 1 1 0 0
1: 0 -> 1 mv=S: 0 1 1 1 1 1 0 0

result: 0 1 1 1 1 1 1 0


In [47]:
UNARY_TIMES_TWO = {
    0: {
        0: (0, 0, "R"), 
        1: (1, 0, "R"),
    },
    1: {
        0: (2, 1, "L"), 
        1: (1, 1, "R"),
    },
    2: {
        0: (3, 0, "R"), 
        1: (4, 0, "R"),
    },
    3: {
        0: (0, 1, "S"), 
        1: (3, 1, "R"),
    },
    4: {
        0: (5, 1, "L"), 
        1: (4, 1, "R"),
    },
    5: {
        0: (2, 1, "L"), 
        1: (5, 1, "L"),
    },
}
unary_times_two = TuringMachine(UNARY_TIMES_TWO)

In [48]:
tape = Tape([0, 1, 1, 1, 0])
tape = Tape([1, 1, 1])
run_turing_machine(unary_times_two, tape)

0: 1 -> 0 mv=R: 1 1 1
1: 1 -> 1 mv=R: 0 1 1
1: 1 -> 1 mv=R: 0 1 1
1: 0 -> 1 mv=L: 0 1 1 0
2: 1 -> 0 mv=R: 0 1 1 1
4: 1 -> 1 mv=R: 0 1 0 1
4: 0 -> 1 mv=L: 0 1 0 1 0
5: 1 -> 1 mv=L: 0 1 0 1 1
5: 0 -> 1 mv=L: 0 1 0 1 1
2: 1 -> 0 mv=R: 0 1 1 1 1
4: 1 -> 1 mv=R: 0 0 1 1 1
4: 1 -> 1 mv=R: 0 0 1 1 1
4: 1 -> 1 mv=R: 0 0 1 1 1
4: 0 -> 1 mv=L: 0 0 1 1 1 0
5: 1 -> 1 mv=L: 0 0 1 1 1 1
5: 1 -> 1 mv=L: 0 0 1 1 1 1
5: 1 -> 1 mv=L: 0 0 1 1 1 1
5: 0 -> 1 mv=L: 0 0 1 1 1 1
2: 0 -> 0 mv=R: 0 1 1 1 1 1
3: 1 -> 1 mv=R: 0 1 1 1 1 1
3: 1 -> 1 mv=R: 0 1 1 1 1 1
3: 1 -> 1 mv=R: 0 1 1 1 1 1
3: 1 -> 1 mv=R: 0 1 1 1 1 1
3: 1 -> 1 mv=R: 0 1 1 1 1 1
3: 0 -> 1 mv=S: 0 1 1 1 1 1 0

result: 0 1 1 1 1 1 1 0


In [49]:
tape = Tape([0, 1, 1, 10])
run_turing_machine(unary_times_two, tape)

0: 0 -> 0 mv=R: 0 1 0
0: 1 -> 0 mv=R: 0 1 0
1: 0 -> 1 mv=L: 0 0 0
2: 0 -> 0 mv=R: 0 0 1
3: 1 -> 1 mv=R: 0 0 1
3: 0 -> 1 mv=S: 0 0 1 0

result: 0 0 1 1 0
